## Tarea 6 - Inteligencia Artificial 
- Andrés Jiménez Mora - Moisés Salguero Morales


### Instalación de librería NEAT

Se instala la librería con la que se trabajará, corriendo el siguiente código 

In [3]:
pip install git+https://github.com/SirBob01/NEAT-Python.git

  Cloning https://github.com/SirBob01/NEAT-Python.git to /tmp/pip-req-build-plzqkc72
  Running command git clone -q https://github.com/SirBob01/NEAT-Python.git /tmp/pip-req-build-plzqkc72


### Importación de Librerías 

Se importan todas las librerías a utilizar

In [4]:
import os
import math
from neat import neat
from numpy import argmax
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


### Importación de datos a evaluar

En esta sección se utilizó la red neuronal creada en la Tarea 1. Que trata saber hacia dónde se movió un robot dependiendo de los datos de sus sensores. Se utilizó el mismo código para obtener una precisión de este modelo. 

In [35]:
# importación de datos 
path = "https://raw.githubusercontent.com/Vektor1428/Inteligencia_Artificial/main/sensor_readings_4.csv"
df = read_csv(path, header=None)

# Definición de entradas y salidas
X, y = df.values[:, :-1], df.values[:, -1]

# asegurarse que los datos sean flotantes
X = X.astype('float32')

# convertir los strings a enteros 
y = LabelEncoder().fit_transform(y)
print(y[0:10])

# Dividir datos en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 5)
#print(X_train[0:10])

# Encontrar el numero de datos de entrada
n_features = X_train.shape[1]

[3 3 3 3 3 3 3 3 1 1]


### Definición del mejor modelo encontrado en la tarea 1
Acá se define manualmente el modelo de la red neuronal con 1 capa oculta de 10 neuronas, 4 entradas y 4 salidas.

In [6]:
# Definir  el modelo 
model = Sequential() # definir el tipo de modelo, en este caso secuencial
model.add(Dense(10, activation='tanh', kernel_initializer='he_normal', input_shape=(n_features,))) #capa oculta
model.add(Dense(4, activation='softmax')) # capa final 

### Evaluación del mejor modelo encontrado en la tarea 1

En esta sección se evalúa el modelo de Red Neuronal mencionado anteriormente

In [7]:
# función  de optimización adam y sus parámetros, entre ellos la tasa de aprendizaje
tf.keras.optimizers.Adam(
    learning_rate=2, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam' )

# compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# entrenar el modelo
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
# evaluar el modelo
loss, acc = model.evaluate(X_test, y_test, verbose=0)

print('Test Accuracy: %.3f' % acc) # imprime la precisión
print ('Test loss: %.3f' %loss)    # imprime la pérdida

Test Accuracy: 0.974
Test loss: 0.131


## Modelo NEAT

### función de calidad

En esta sección se define la función de calidad para el caso del algoritmo NEAT

In [28]:
def fitness(modelo):
    """Calculates the similarity score between expected and output."""
    expected = modelo[1]
    output = modelo[0]
    numero= modelo[2]

    error_total = []
    for i in range(numero):
      error = abs(expected[i] - output[1])
      error_total.append(error)
    
    error_medio = sum(error_total)/numero
    return 1/(1+error_medio)

## Definición de función de evaluación 

La función de evaluación simplemente evalúa las entradas para obtener las salidas que produce la red. Y eso lo dirige hacia la función Fitness para encontrar la calidad de la respuesta

In [30]:
def evaluate(genome):
    salidas = []

    for i in range(2000):
      """Evaluates the current genome."""
      salida1 = genome.forward(X_train[i])[0]
      salida2 = genome.forward(X_train[i])[1]
      salida3 = genome.forward(X_train[i])[2]
      salida4 = genome.forward(X_train[i])[3]
      
      salidas.append(salida1 + salida2 + salida3 + salida4)

    modelo = [salidas, y_train, 2000]
    return fitness(modelo)

### Función Main 

Acá es donde se define principalmente el modelo genético: su población inicial, la cantidad máxima de generaciones, etc.

In [36]:
def main():
    hyperparams = neat.Hyperparameters() 
    hyperparams.max_generations = 100 # cantidad máxima de generaciones
    hyperparams.mutation_probabilities["weight_perturb"] = 0.5 # probablidad de mutación 

    brain = neat.Brain(n_features, n_features, 40, hyperparams) # cantidad de entradas, salidas, población inicial
    brain.generate()
    
    print("Training...")
    while brain.should_evolve(): # ciclo que evalúa constantemente la calidad 
        brain.evaluate_parallel(evaluate)

        # Print training progress
        current_gen = brain.get_generation()
        current_best = brain.get_fittest()
        print("Current Accuracy: {:.2f}% | Generation {}/{}".format(
            current_best.get_fitness() * 100, 
            current_gen, 
            hyperparams.max_generations
        ))

    best = brain.get_fittest()
    salidas = []

    for l in range(1000): # Evaluación de la calidad al mejor individuo 
      """Evaluates the current genome."""
      salida1 = best.forward(X_test[l])[0]
      salida2 = best.forward(X_test[l])[1]
      salida3 = best.forward(X_test[l])[2]
      salida4 = best.forward(X_test[l])[3]
      
      salidas.append(salida1 + salida2 + salida3 + salida4)
    modelo = [salidas, y_test, len(y_test)]
    fit = fitness(modelo) # calidad del mejor individuo 


    edges = best.get_edges()
    graph = {}
    for (i, j) in edges:
        if not edges[(i, j)].enabled:
            continue
        if i not in graph:
            graph[i] = []
        graph[i].append(j)

    print()
    print(f"Best network structure: {best.get_num_nodes()} nodes")
    for k in graph:
        print(f"{k} - {graph[k]}")
    print()
    print("Accuracy: {:.2f}%".format(fit * 100))

### Llamada a main 

In [37]:
if __name__ == "__main__":
    main()

Training...
Current Accuracy: 41.97% | Generation 1/100
Current Accuracy: 56.87% | Generation 2/100
Current Accuracy: 56.89% | Generation 3/100
Current Accuracy: 56.89% | Generation 4/100
Current Accuracy: 56.89% | Generation 5/100
Current Accuracy: 56.96% | Generation 6/100
Current Accuracy: 56.96% | Generation 7/100
Current Accuracy: 56.96% | Generation 8/100
Current Accuracy: 56.96% | Generation 9/100
Current Accuracy: 56.96% | Generation 10/100
Current Accuracy: 56.96% | Generation 11/100
Current Accuracy: 56.96% | Generation 12/100
Current Accuracy: 56.96% | Generation 13/100
Current Accuracy: 56.96% | Generation 14/100
Current Accuracy: 56.96% | Generation 15/100
Current Accuracy: 56.96% | Generation 16/100
Current Accuracy: 56.96% | Generation 17/100
Current Accuracy: 56.96% | Generation 18/100
Current Accuracy: 56.96% | Generation 19/100
Current Accuracy: 56.96% | Generation 20/100
Current Accuracy: 56.96% | Generation 21/100
Current Accuracy: 56.96% | Generation 22/100
Current

### Análisis de resultados
Como se puede observar el modelo NEAT que se obtuvo no tuvo una precisión adecuada, eso se puede deber a las pocas pruebas realizadas. En este ejemplo se realizaron pruebas con pocas generaciones y poca población inicial debido a que los equipos computacionales no tienen capacidades áltas de trabajo. Esta es una de las razones por las que se pudo hacer obtenido resultados bajos. Como se observa el algoritmo NEAT obtuvo una precisión de 56.43% mientras que el algoritmo que se creó con Keras, con 1 capa oculta y 10 neuronas en su capa oculta, tuvo una precisión del 97.4% esto es mucho mayor a lo que se obtuv con NEAT. 

Como conclusión, se deben tener equipos que puedan procesar datos a altas velocidades y tengan una gran capacidad computacional para obtener buenos resultados en tiempos aceptables. Si bien es cierto que el algotimo NEAT funciona muy bien, su costo computacional es bastante elevado. 